In [1]:
import pandas as pd
import numpy as np
from dsipts import TimeSeries, RNN,read_public_dataset, LinearTS, Persistent, TFT
import matplotlib.pyplot as plt
from datetime import timedelta
import logging
import sys
import random



In [18]:
def random_walk(n):
    tot = np.zeros(n)
    probs = np.zeros(n)
    for i in range(n-1):
        prob = random.random()
        if prob<0.5:
            delta = 1
        else:
            delta = -1
        tot[i+1] = tot[i] + delta
        probs[i+1]= prob
    return tot, probs
N = 20000
random.seed(6)
x, p = random_walk(N)
data = pd.DataFrame({'y':x/x.max(),'p':p,'time':range(N)})

In [19]:
##loasklearnthe timeseries to the datastructure, adding the hour column and use all the covariates
ts = TimeSeries('weather')
ts.load_signal(data,enrich_cat=[],target_variables=['y'],past_variables= [],future_variables=['p'])
ts

Timeseries named weather of length 20000.
 Categorical variable: [],
 Future variables: ['p'],
 Past variables: ['y'],
 Target variables: ['y']
 With no group

In [20]:
#Let now prepare a model predictin the next 16 step using the past 16 steps 
past_steps = 1
future_steps = 16


In [21]:
ts.future_variables

['p']

In [35]:


config = dict(model_configs =dict(
                                    past_steps = past_steps,
                                    future_steps = future_steps,
                                    past_channels = len(ts.past_variables),
                                    future_channels = len(ts.future_variables),
                                    embs = [ts.dataset[c].nunique() for c in ts.cat_var],
                                      d_model=64,
                                      d_head= 6,
                                      n_head=16,
                                     dropout_rate = 0.1,
                                      num_layers_RNN= 8,
                                      optim='torch.optim.Adam',
                                     out_channels = len(ts.target_variables),
                                    quantiles= [],
                                   loss_type='mse',
                                 persistence_weight = 4,
                                    ),
                scheduler_config = dict(gamma=0.1,step_size=2400000000000),
                optim_config = dict(lr = 0.00005,weight_decay=0.0))
model_linear = TFT(**config['model_configs'],optim_config = config['optim_config'],scheduler_config =config['scheduler_config'],verbose=False )


In [36]:
#set the desirere model
ts.set_model(model_linear,config=config )

In [37]:
##splitting parameters
split_params = {'perc_train':0.6,'perc_valid':0.2,                             ##if not None it will split 70% 10% 20%
               'range_train':None, 'range_validation':None, 'range_test':None, ## or we can split using ranges for example range_train=['2021-02-03','2022-04-08']
               'past_steps':past_steps,
               'future_steps':future_steps,
               'shift':0,
               'starting_point':None,                                          ## do not skip samples
               'skip_step' : 1                                                 ## distance between two consecutive samples
                             }

In [ ]:
#train the model for 50 epochs with auto_lr_find 
ts.train_model(dirpath=f"/home/agobbi/Projects/ExpTS/rf/tft",
               split_params=split_params,
               batch_size=32,
               num_workers=2,
               max_epochs=25,
               auto_lr_find=False)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/agobbi/.conda/envs/tt/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/agobbi/Projects/ExpTS/rf/tft exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

   | Name                   | Type                    | Params
--------------------------------------------------------------------
0  | target_linear          | Linear                  | 128   
1  | linear_aux_past        | ModuleList              | 0     
2  | linear_aux_fut         | ModuleList              | 128   
3  | emb_cat_var            | embedding_cat_variables | 2.3 K 
4  | rnn                    | LSTM_Model              | 267 K 
5  | res_conn1_past         | ResidualConnection      | 8.3 K 
6  | res_conn1_fut          | ResidualC

In [26]:
#Print the losses, check overfitting
ts.losses.plot()

<Axes: >

In [27]:
#make inferences on 
res = ts.inference_on_set(200,4,set='test',rescaling=True)

In [29]:
%matplotlib qt
lag = 15

plt.plot(res[res.lag==lag].time, res[res.lag==lag].y,label='real',alpha=0.5)
plt.plot(res[res.lag==lag].time, res[res.lag==lag].y_pred,label='pred',alpha=0.5)

plt.title('Prediction on test for lag=7')
plt.legend()

In [30]:
res['prediction_time'] = res.apply(lambda x: int(x.time-x.lag), axis=1)

In [31]:
res

,lag,time,y,y_pred,prediction_time
0,1,16001,0.370079,0.361578,16000
1,1,16002,0.362205,0.359649,16001
2,1,16003,0.370079,0.364952,16002
3,1,16004,0.377953,0.377869,16003
4,1,16005,0.385827,0.381918,16004
...,...,...,...,...,...
63723,16,19994,0.771654,0.803363,19978
63724,16,19995,0.763780,0.807920,19979
63725,16,19996,0.755906,0.812142,19980
63726,16,19997,0.748031,0.804506,19981


In [34]:
%matplotlib qt
date = 19679

plt.plot(res[res.prediction_time==date].time, res[res.prediction_time==date].y,label='real',alpha=0.5)
plt.plot(res[res.prediction_time==date].time, res[res.prediction_time==date].y_pred,label='pred',alpha=0.5)
#plt.ylim(res.y.min(),res.y.max())
plt.title('Prediction on test for lag=7')
plt.legend()